In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0" # define GPU id, remove if you want to use all GPUs available
import torch
from tqdm import tqdm
import time
from contextlib import contextmanager
import numpy as np
import transformers
from huggingface_hub import hf_hub_download

c:\Users\admin\Downloads\Medusa-main\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import torch
import time
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "lmsys/vicuna-7b-v1.3"  
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

prompt = "A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. USER: Hi, could you share a tale about a charming llama that grows Medusa-like hair and starts its own coffee shop? ASSISTANT:"

input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")

_ = model.generate(input_ids, max_length=20)

torch.cuda.synchronize() 
start_time = time.time()

output = model.generate(input_ids, max_length=50, do_sample=True)
torch.cuda.synchronize() 

end_time = time.time()

num_generated_tokens = output.size(-1) - input_ids.size(-1)  
elapsed_time = end_time - start_time
tokens_per_second = num_generated_tokens / elapsed_time

print(f"Generated {num_generated_tokens} tokens in {elapsed_time:.2f} seconds.")
print(f"Tokens per second: {tokens_per_second:.2f}")


c:\Users\admin\Downloads\Medusa-main\.venv\Lib\site-packages\accelerate\utils\modeling.py:1390: UserWarning: Current model requires 32509824 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.71s/it]
Some parameters are on the meta device because they were offloaded to the cpu.
c:\Users\admin\Downloads\Medusa-main\.venv\Lib\site-packages\transformers\generation\utils.py:1268: UserWarning: Input length of input_ids is 66, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(
c:\Users\admin\Downloads\Medusa-main\.venv\Lib\site-packages\transformers\generation\utils.py:1268: UserWarning: Input length of input_ids is 66, but `max_length` is set to 50. This can lead to unexpected behavior. You should consider in

Generated 1 tokens in 1.61 seconds.
Tokens per second: 0.62


In [3]:
import torch
import time
from transformers import AutoModelForCausalLM, AutoTokenizer

# Set model path and configurations
model_name = "lmsys/vicuna-7b-v1.3"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)

# Load the model with GPU allocation using device_map and low_cpu_mem_usage=True
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",  # Automatically maps model to available devices
    low_cpu_mem_usage=True,  # Required when using device_map
)

# Define a sample prompt
prompt = "A chat between a curious user and an artificial intelligence assistant."

# Tokenize the input prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")  # Ensure tensors are on GPU

# Warmup generation (to load the model into memory)
_ = model.generate(input_ids, max_length=20)

# Measure inference time for performance benchmarking
torch.cuda.synchronize()  # Ensure all GPU operations are finished
start_time = time.time()

# Perform text generation with the model
output = model.generate(input_ids, max_length=50, do_sample=True)

torch.cuda.synchronize()  # Ensure the model finishes generating
end_time = time.time()

# Calculate tokens per second
num_generated_tokens = output.size(-1) - input_ids.size(-1)  # Exclude input tokens
elapsed_time = end_time - start_time
tokens_per_second = num_generated_tokens / elapsed_time

# Print benchmarking results
print(f"Generated {num_generated_tokens} tokens in {elapsed_time:.2f} seconds.")
print(f"Tokens per second: {tokens_per_second:.2f}")


c:\Users\admin\Downloads\Medusa-main\.venv\Lib\site-packages\accelerate\utils\modeling.py:1390: UserWarning: Current model requires 32509824 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.57s/it]
Some parameters are on the meta device because they were offloaded to the cpu.


Generated 37 tokens in 59.69 seconds.
Tokens per second: 0.62
